In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
ticker_list = stock.get_market_ticker_list(market="KOSPI")

# precision_svm 값과 종목코드를 저장할 빈 리스트
results_list = []

# 각 종목에 대해 반복
for ticker in tqdm(ticker_list, desc="Processing"):
    try:
        ticker_name = stock.get_market_ticker_name(ticker)
        start_date = "2021-03-20"
        end_date = "2024-03-20"
        # 주어진 기간 동안의 일별 거래량 정보 가져오기
        df = stock.get_market_ohlcv_by_date(fromdate=start_date, todate=end_date, ticker=ticker)


        def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
            """MACD 및 MACD 신호 계산"""
            df['EMA_short'] = df['종가'].ewm(span=short_window, adjust=False).mean()
            df['EMA_long'] = df['종가'].ewm(span=long_window, adjust=False).mean()
            df['MACD'] = df['EMA_short'] - df['EMA_long']
            df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
            return df

        def calculate_rsi(df, window=14, signal_window=9):
            """RSI 및 RSI 신호 계산"""
            delta = df['종가'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

            rs = gain / loss
            df['RSI'] = 100 - (100 / (1 + rs))
            
            # RSI 신호선 추가
            df['RSI_Signal'] = df['RSI'].ewm(span=signal_window, adjust=False).mean()
            return df

        # MACD 및 MACD 신호 계산
        df = calculate_macd(df)

        df = calculate_rsi(df)

        # '종가' 컬럼이 포함된 DataFrame을 가정합니다. 예를 들어, df라고 합시다.
        # df = pd.read_csv('path_to_your_data.csv') # 데이터 파일을 로드하는 예시

        # 지정된 기간에 대한 SMA 계산
        periods = [5, 20, 60, 120, 240]
        for period in periods:
            df[f'SMA_{period}'] = df['종가'].rolling(window=period).mean()

        # 지정된 기간에 대한 EMA 계산
        for period in periods:
            df[f'EMA_{period}'] = df['종가'].ewm(span=period, adjust=False).mean()

        for i in range(len(periods)):
            for j in range(i + 1, len(periods)):
                smaller_period = periods[i]
                larger_period = periods[j]
                df[f'SMA_{smaller_period}_minus_SMA_{larger_period}'] = df[f'SMA_{smaller_period}'] - df[f'SMA_{larger_period}']

        # EMA 간의 차이 계산
        for i in range(len(periods)):
            for j in range(i + 1, len(periods)):
                smaller_period = periods[i]
                larger_period = periods[j]
                df[f'EMA_{smaller_period}_minus_EMA_{larger_period}'] = df[f'EMA_{smaller_period}'] - df[f'EMA_{larger_period}']

        for column in ['시가', '고가', '저가','거래량','MACD','MACD_Signal','RSI','RSI_Signal']:
            df[f'{column}_등락률'] = df[column].pct_change() * 100

        # SMA 및 EMA의 전 거래일 대비 등락률 계산 및 DataFrame에 추가
        periods = [5, 20, 60, 120, 240]

        # SMA 등락률 계산 및 추가
        for period in periods:
            df[f'SMA_{period}_등락률'] = df[f'SMA_{period}'].pct_change() * 100

        # EMA 등락률 계산 및 추가
        for period in periods:
            df[f'EMA_{period}_등락률'] = df[f'EMA_{period}'].pct_change() * 100

        # 'MACD'가 0 이상일 때 1, 아니면 0을 할당
        df['MACD_Binary'] = (df['MACD'] >= 0).astype(int)

        # 'MACD - MACD_Signal'이 0 이상일 때 1, 아니면 0을 할당
        df['MACD_minus_Signal_Binary'] = ((df['MACD'] - df['MACD_Signal']) >= 0).astype(int)

        # 'RSI - RSI_Signal'이 0 이상일 때 1, 아니면 0을 할당
        df['RSI_minus_Signal_Binary'] = ((df['RSI'] - df['RSI_Signal']) >= 0).astype(int)

        # 시가 대비 종가 등락률 컬럼 추가
        df['시가_대비_종가_등락률'] = ((df['종가'] - df['시가']) / df['시가']) * 100
        df['시가_대비_저가_등락률'] = ((df['저가'] - df['시가']) / df['시가']) * 100
        df['시가_대비_고가_등락률'] = ((df['고가'] - df['시가']) / df['시가']) * 100
        df['저가_대비_종가_등락률'] = ((df['종가'] - df['저가']) / df['저가']) * 100
        df['저가_대비_고가_등락률'] = ((df['고가'] - df['저가']) / df['저가']) * 100
        df['고가_대비_종가_등락률'] = ((df['종가'] - df['고가']) / df['고가']) * 100
        # 종가 - SMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
        for period in [5, 20, 60, 120, 240]:
            df[f'종가_minus_SMA_{period}'] = df['종가'] - df[f'SMA_{period}']

        # 종가 - EMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
        for period in [5, 20, 60, 120, 240]:
            df[f'종가_minus_EMA_{period}'] = df['종가'] - df[f'EMA_{period}']


        #csv_file_path = "C:/apps/h1/이스트소프트_data.csv"  # 저장할 파일 경로 및 이름 설정
        #df.to_csv(csv_file_path, encoding='utf-8-sig')

        #print(f"Data saved to {csv_file_path}")

        # 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
        df['next_day_return'] = df['등락률'].shift(-1)
        df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else -1)
        df.dropna(inplace=True)  # 마지막 행 삭제

        first_column_name = df.columns[0]

        df1= df.iloc[:-5, :]
        X = df1.drop(['next_day_return','target'], axis=1)
        y = df1['target']

        # Calculating correlations for Forward Selection
        correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
        correlations.columns = ['Feature', 'Correlation']

        #top_8_features = correlations.sort_values(by='Correlation', ascending=False).head(8)['Feature']
        #top8_correlation = correlations.sort_values(by='Correlation', ascending=False).head(8)['Correlation']
        highest_correlation=correlations.sort_values(by='Correlation', ascending=False).head(1)['Correlation']
        results_list.append((ticker, ticker_name, highest_correlation ))

        # 결과를 데이터프레임으로 변환
        results_df = pd.DataFrame(results_list, columns=['ticker', 'ticker_name', 'highest_correlation'])
        # correlations 데이터프레임의 상위 8개 상관계수 값을 선택
        top8_correlation = correlations['Correlation'].iloc[:8].reset_index(drop=True)

        # 새로운 열 이름을 생성 (Top1_Correlation, Top2_Correlation, ..., Top8_Correlation)
        new_columns = [f"Top{i+1}_Correlation" for i in range(8)]

        # 각 상관계수 값을 개별 열로 변환하여 새로운 데이터프레임 생성
        top8_correlation_df = pd.DataFrame(top8_correlation.values.reshape(1, 8), columns=new_columns)

        # 기존 데이터프레임(df)에 새로운 데이터프레임(top8_correlation_df)을 가로로(concatenate horizontally) 붙임
        # 이 예시에서는 df가 이미 존재한다고 가정합니다.
        df_with_top8 = pd.concat([results_df, top8_correlation_df], axis=1)
        pass
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue

# # 최고 상관계수로 내림차순 정렬
# results_df_sorted = df_with_top8.sort_values(by='highest_correlation', ascending=False).reset_index(drop=True)



Processing: 100%|██████████| 953/953 [06:38<00:00,  2.39it/s]


In [15]:
df_with_top8.head()

,ticker,ticker_name,highest_correlation,Top1_Correlation,Top2_Correlation,Top3_Correlation,Top4_Correlation,Top5_Correlation,Top6_Correlation,Top7_Correlation,Top8_Correlation
0,095570,AJ네트웍스,"0 0.07845 Name: Correlation, dtype: float64",0.111389,0.068097,0.066021,0.063416,0.059762,0.059271,0.0575,0.05612
1,006840,AK홀딩스,"0 0.135792 Name: Correlation, dtype: float64",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,027410,BGF,"0 0.127063 Name: Correlation, dtype: float64",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,282330,BGF리테일,"0 0.136616 Name: Correlation, dtype: float64",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,138930,BNK금융지주,"0 0.107775 Name: Correlation, dtype: float64",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
csv_file_path = "C:/apps/h1/kospi_corr_data.csv"  # 저장할 파일 경로 및 이름 설정
df_with_top8.to_csv(csv_file_path, encoding='utf-8-sig')